# Welcome to czapi

> A basic API for scraping curling boxscores off of the popular [CurlingZone](https://www.curlingzone.com) website. 

## Install

```
pip install czapi
```

## How to use

In [9]:
import czapi.api as api

## LinescorePage
czapi is based around the `LinescorePage` object which represents a linescore page on the CurlingZone website. 

Click [here](https://curlingzone.com/event.php?view=Scores&eventid=7795#1) to see an example linescore page.

Creating an instance of the `LinescorePage` class gives access to every boxscore on that linescore page.

In [10]:
linescore_page = api.LinescorePage(cz_event_id = 6400, cz_draw_id = 2)

The `cz_event_id` and `cz_draw_id` arguments are found in the CurlingZone URL. 

Here's an example:


> ht<span>t</span>ps://curlingzone.com/event.php?**eventid=7795**&view=Scores&show**drawid=21**#1

The boxscores on the linescore page can be accessed through the `boxscores` property which returns a list of boxscores.

In [11]:
linescore_page.boxscores

[{'Wayne Tuck Jr.': {'href': 'event.php?view=Team&eventid=6400&teamid=144353&profileid=12486#1',
   'score': ['0', '2', '0', '0', '0', '0', '1', '1', '1', '0'],
   'hammer': True,
   'finalscore': '5'},
  'Matthew Hall': {'href': 'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1',
   'score': ['0', '0', '4', '0', '0', '1', '0', '0', '0', '2'],
   'hammer': False,
   'finalscore': '7'}},
 {'Dayna Deruelle': {'href': 'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435&eventid=6400&teamid=144346&profileid=26636#1',
   'score': ['0', '0', '1', '0', '0', '0', '0', 'X'],
   'hammer': False,
   'finalscore': '1'},
  'Tyler Stewart': {'href': 'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435&eventid=6400&teamid=144352&profileid=12477#1',
   'score': ['0', '2', '0', '2', '1', '1', '4', 'X'],
   'hammer': True,
   'finalscore': '10'}},
 {'Mark Kean': {'href': 'event.php?view=Team&eventid=6400&teamid=144352&profileid=12477&eventid=6400&teamid=144348&profi

A boxscore is returned as a dictionary of team names, game information dictionary key, value pairs.

Each game information dictionary contains:

* 'href' key with a corresponding string value of CurlingZone IDs identifying the team.
* 'hammer' key with corresponding boolean value of whether or not that team started the game with hammer.
* 'score' key with corresponding list of string value of end-by-end results for that team.
* 'finalscore' key with corresponding final score for the team.

Individual boxscores can be accessed through the `get_boxscore_from` method.

In [12]:
boxscore = linescore_page.get_boxscore_from(cz_game_id = 1)
boxscore

{'Wayne Tuck Jr.': {'href': 'event.php?view=Team&eventid=6400&teamid=144353&profileid=12486#1',
  'score': ['0', '2', '0', '0', '0', '0', '1', '1', '1', '0'],
  'hammer': True,
  'finalscore': '5'},
 'Matthew Hall': {'href': 'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1',
  'score': ['0', '0', '4', '0', '0', '1', '0', '0', '0', '2'],
  'hammer': False,
  'finalscore': '7'}}

`cz_game_id` argument corresponds to the number the boxscore appears in on the linescore page.

The LinescorePage object contains these properties which details information on the boxscores:

* event_name
* event_date
* draw

In [13]:
print(linescore_page.event_name,',',linescore_page.event_date,',' ,linescore_page.draw)

Ontario Tankard - Open Qualifier , Jan 17 - 19, 2020 , Draw: 2


For convenience, upon instantiation of a `LinescorePage` object, a `NormalizedBoxscore` instance for each boxscore is created. A `NormalizedBoxscore` holds the same information as a boxscore dictionary with two new pieces of information added: 
1. The hammer progression for both teams throughout the game, i.e. who had hammer in what end.
2. Each team's relative score, i.e. who was up/down X after end Y.

In [14]:
normalized_boxscore = linescore_page.get_normalized_boxscore_from(cz_game_id = 1)
normalized_boxscore

NormalizedBoxscore(boxscore={'Wayne Tuck Jr.': {'href': 'event.php?view=Team&eventid=6400&teamid=144353&profileid=12486#1', 'score': ['0', '2', '0', '0', '0', '0', '1', '1', '1', '0'], 'hammer': True, 'finalscore': '5'}, 'Matthew Hall': {'href': 'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1', 'score': ['0', '0', '4', '0', '0', '1', '0', '0', '0', '2'], 'hammer': False, 'finalscore': '7'}})

A `NormalizedBoxscore` object holds two `NormalizedHalfBoxscore` instances. 

In [15]:
normalized_boxscore.normalized_half_boxscore_pair[0]

NormalizedHalfBoxscore(team_name='Wayne Tuck Jr.', href='event.php?view=Team&eventid=6400&teamid=144353&profileid=12486#1', hammer=True, score=['0', '2', '0', '0', '0', '0', '1', '1', '1', '0'], finalscore='5', hammer_progression=[True, True, False, True, True, True, True, False, False, False], normalized_score=[0, 0, 2, -2, -2, -2, -3, -2, -1, 0])

For Wayne Tuck Jr. the `hammer_progression` attribute can be interpreted as follows: 

* End 1: Wayne had hammer
* End 2: Wayne had hammer
* End 3: Wayne didn't have hammer
* And so on and so forth..

In [16]:
normalized_boxscore.normalized_half_boxscore_pair[-1]

NormalizedHalfBoxscore(team_name='Matthew Hall', href='event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1', hammer=False, score=['0', '0', '4', '0', '0', '1', '0', '0', '0', '2'], finalscore='7', hammer_progression=[False, False, True, False, False, False, False, True, True, True], normalized_score=[0, 0, -2, 2, 2, 2, 3, 2, 1, 0])

For Matthew Hall, the `normalized_score` attribute can be interpreted as follows:

* The score was tied in the first end (obviously).
* The score was tied in the second end.
* In the 3rd end, Matthew was down 2.
* In the 4th end, Matthew was up 2 after taking 4.
* And so on and so forth..

You'll also notice the `NormalizedBoxscore` object has a guid property which identifies that two `NormalizedHalfBoxscore` belong to the same game.

In [17]:
normalized_boxscore.guid

138077201045659598503586885217589832745

czapi's `get_flat_boxscores_from` function takes a `cz_event_id` and `cz_draw_id` as an arguments and returns a (flat) nested list object of all the boxscore information on the linescore page. This nested list object can be ingested into a pandas DataFrame or pushed to a SQL database.

In [18]:
api.get_flat_boxscores_from(cz_event_id = 6400, cz_draw_id = 2)

[('Wayne Tuck Jr.',
  'event.php?view=Team&eventid=6400&teamid=144353&profileid=12486#1',
  True,
  ['0', '2', '0', '0', '0', '0', '1', '1', '1', '0'],
  '5',
  [True, True, False, True, True, True, True, False, False, False],
  [0, 0, 2, -2, -2, -2, -3, -2, -1, 0],
  303512780053662978549921974368827460012),
 ('Matthew Hall',
  'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1',
  False,
  ['0', '0', '4', '0', '0', '1', '0', '0', '0', '2'],
  '7',
  [False, False, True, False, False, False, False, True, True, True],
  [0, 0, -2, 2, 2, 2, 3, 2, 1, 0],
  303512780053662978549921974368827460012),
 ('Dayna Deruelle',
  'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435&eventid=6400&teamid=144346&profileid=26636#1',
  False,
  ['0', '0', '1', '0', '0', '0', '0', 'X'],
  '1',
  [False, False, True, False, True, True, True, True],
  [0, 0, -2, -1, -3, -4, -5, -9],
  24327381068296329695831492444794887720),
 ('Tyler Stewart',
  'event.php?view=Team&eventid=6400&te

## Event

The `Event` object is a data structure which holds all of the `LinescorePage` objects which make up an entire event. 

An `Event` instance is created by passing a `cz_event_id`.

In [19]:
event = api.Event(cz_event_id = 6400,delay=3,verbose=True)
event

Scraping draw 1.
Scraping draw 2.
Scraping draw 3.
Scraping draw 4.
Scraping draw 5.
Scraping draw 6.
Scraping draw 7.


Event(cz_event_id=6400, delay=3, verbose=True)

The created `Event` objects holds all the `LinescorePage` objects in it's `pages` property.

In [20]:
event.pages

[LinescorePage(cz_event_id=6400, cz_draw_id=1),
 LinescorePage(cz_event_id=6400, cz_draw_id=2),
 LinescorePage(cz_event_id=6400, cz_draw_id=3),
 LinescorePage(cz_event_id=6400, cz_draw_id=4),
 LinescorePage(cz_event_id=6400, cz_draw_id=5),
 LinescorePage(cz_event_id=6400, cz_draw_id=6),
 LinescorePage(cz_event_id=6400, cz_draw_id=7)]

Details about specific draws can be accessed by grabbing the correct LinescorePage.

In [21]:
event.pages[2]

LinescorePage(cz_event_id=6400, cz_draw_id=3)

In [22]:
event.pages[2].boxscores

[{'Matthew Hall': {'href': 'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1',
   'score': ['0', '0', '2', '1', '2', '1', '0', '2', 'X'],
   'hammer': True,
   'finalscore': '8'},
  'Tyler Stewart': {'href': 'event.php?view=Team&eventid=6400&teamid=144352&profileid=12477#1',
   'score': ['0', '1', '0', '0', '0', '0', '2', '0', 'X'],
   'hammer': False,
   'finalscore': '3'}},
 {'Mark Kean': {'href': 'event.php?view=Team&eventid=6400&teamid=144352&profileid=12477&eventid=6400&teamid=144348&profileid=25961#1',
   'score': ['0', '5', '0', '1', '1', '1', 'X'],
   'hammer': True,
   'finalscore': '8'},
  'Richard Krell': {'href': 'event.php?view=Team&eventid=6400&teamid=144352&profileid=12477&eventid=6400&teamid=144349&profileid=25962#1',
   'score': ['1', '0', '1', '0', '0', '0', 'X'],
   'hammer': False,
   'finalscore': '2'}},
 {'Damien Villard': {'href': 'event.php?view=Team&eventid=6400&teamid=144349&profileid=25962&eventid=6400&teamid=144354&profileid=27373#1',
   'sco

The `get_flat_boxscores` method can be used to return a list of (flat) nested list object of all the boxscore information on all the linescore pages.

In [23]:
event.get_flat_boxscores()

[[('Damien Villard',
   'event.php?view=Team&eventid=6400&teamid=144354&profileid=27373#1',
   True,
   ['0', '0', '0', '0', 'X'],
   '0',
   [True, True, True, True, True],
   [0, -1, -4, -6, -8],
   248919024004949934716914821388389967307),
  ('Matthew Hall',
   'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1',
   False,
   ['1', '3', '2', '2', 'X'],
   '8',
   [False, False, False, False, False],
   [0, 1, 4, 6, 8],
   248919024004949934716914821388389967307),
  ('Matthew Mepstead',
   'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435&eventid=6400&teamid=144356#1',
   False,
   ['0', '0', '1', '1', '0', '1', '0', '0', '0', '1', '0'],
   '4',
   [False, False, True, False, False, True, False, False, False, True, False],
   [0, 0, -2, -1, 0, -1, 0, 0, 0, -1, 0],
   142381201445779913776545786214530950693),
  ('Tyler Stewart',
   'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435&eventid=6400&teamid=144352&profileid=12477#1',
   True,
   ['0

In [24]:
event.get_flat_boxscores()[0]

[('Damien Villard',
  'event.php?view=Team&eventid=6400&teamid=144354&profileid=27373#1',
  True,
  ['0', '0', '0', '0', 'X'],
  '0',
  [True, True, True, True, True],
  [0, -1, -4, -6, -8],
  248919024004949934716914821388389967307),
 ('Matthew Hall',
  'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1',
  False,
  ['1', '3', '2', '2', 'X'],
  '8',
  [False, False, False, False, False],
  [0, 1, 4, 6, 8],
  248919024004949934716914821388389967307),
 ('Matthew Mepstead',
  'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435&eventid=6400&teamid=144356#1',
  False,
  ['0', '0', '1', '1', '0', '1', '0', '0', '0', '1', '0'],
  '4',
  [False, False, True, False, False, True, False, False, False, True, False],
  [0, 0, -2, -1, 0, -1, 0, 0, 0, -1, 0],
  142381201445779913776545786214530950693),
 ('Tyler Stewart',
  'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435&eventid=6400&teamid=144352&profileid=12477#1',
  True,
  ['0', '2', '0', '0', '1', '0', 

In [25]:
event.get_flat_boxscores()[-1]

[('Tyler Stewart',
  'event.php?view=Team&eventid=6400&teamid=144352&profileid=12477#1',
  False,
  ['0', '0', '1', '1', '0', '3', '0', '0', '1', '0'],
  '6',
  [False, False, True, False, False, True, False, False, True, False],
  [0, 0, -1, 0, 1, -2, 1, 1, 0, 1],
  141376204131397514047801901828601390592),
 ('Matthew Hall',
  'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1',
  True,
  ['0', '1', '0', '0', '3', '0', '0', '1', '0', '2'],
  '7',
  [True, True, False, True, True, False, True, True, False, True],
  [0, 0, 1, 0, -1, 2, -1, -1, 0, -1],
  141376204131397514047801901828601390592)]

## About czapi
czapi is a Python library for scraping curling linescores.